In [8]:
import os
import time
import argparse

import torchvision
import torch
import torch.nn as nn

from util import AverageMeter
from encoder import SmallAlexNet
from align_uniform import align_loss, uniform_loss_prelog
from tqdm import tqdm
from collections import defaultdict
import numpy as np
import copy

import matplotlib.pyplot as plt


class TwoAugUnsupervisedDatasetLbl(torch.utils.data.Dataset):
    r"""Returns two augmentation and no labels."""

    def __init__(self, dataset, transform, lblmap=None):
        self.dataset = dataset
        # load the images on GPU
        self.dataset= dataset
        self.transform = transform
        self.lblmap = copy.deepcopy(lblmap)

    def __getitem__(self, index):
        image, lbl = self.dataset[index]
        lbl2return = lbl if self.lblmap is None else self.lblmap[lbl]
        return self.transform(image), self.transform(image), lbl2return

    def __len__(self):
        return len(self.dataset)

def parse_option():
    
    parser = argparse.ArgumentParser('STL-10 Representation Learning with Alignment and Uniformity Losses')

    parser.add_argument('--align_w', type=float, default=1, help='Alignment loss weight')
    parser.add_argument('--unif_w', type=float, default=1, help='Uniformity loss weight')
    parser.add_argument('--align_alpha', type=float, default=2, help='alpha in alignment loss')
    parser.add_argument('--unif_t', type=float, default=2, help='t in uniformity loss')

    parser.add_argument('--batch_size', type=int, default=768, help='Batch size')
    parser.add_argument('--epochs', type=int, default=200, help='Number of training epochs')
    parser.add_argument('--iter', type=int, default=0, help='Number of training epochs')
    parser.add_argument('--lr', type=float, default=None,
                        help='Learning rate. Default is linear scaling 0.12 per 256 batch size')
    parser.add_argument('--lr_decay_rate', type=float, default=0.1, help='Learning rate decay rate')
    parser.add_argument('--lr_decay_epochs', default=[155,175,185], nargs='*', type=int,
                        help='When to decay learning rate')
    parser.add_argument('--momentum', type=float, default=0.9, help='SGD momentum')
    parser.add_argument('--weight_decay', type=float, default=1e-4, help='L2 weight decay')
    parser.add_argument('--feat_dim', type=int, default=128, help='Feature dimensionality')

    parser.add_argument('--num_workers', type=int, default=80, help='Number of data loader workers to use')
    parser.add_argument('--log_interval', type=int, default=40, help='Number of iterations between logs')
    parser.add_argument('--gpus', default=[0], nargs='*', type=int,
                        help='List of GPU indices to use, e.g., --gpus 0 1 2 3')

    parser.add_argument('--data_folder', type=str, default='./data', help='Path to data')
    parser.add_argument('--result_folder', type=str, default='./results', help='Base directory to save model')

    opt = parser.parse_args("")

    if opt.lr is None:
        opt.lr = 0.12 * (opt.batch_size / 256)

    opt.gpus = list(map(lambda x: torch.device('cuda', x), opt.gpus))

    opt.save_folder = os.path.join(
        opt.result_folder,
        f"cifar100_supervised_170_2_oldnewnewset_smllr_{opt.feat_dim}_{opt.iter}_{opt.epochs}"
    )
    os.makedirs(opt.save_folder, exist_ok=True)

    return opt


opt = parse_option()

In [9]:
opt.gpus[0]

device(type='cuda', index=0)

In [10]:
"""
transform = torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(32, scale=(0.08, 1)),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ColorJitter(0.4, 0.4, 0.4, 0.4),
        torchvision.transforms.RandomGrayscale(p=0.2),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.44087801806139126, 0.42790631331699347, 0.3867879370752931),
            (0.26826768628079806, 0.2610450402318512, 0.26866836876860795),
        ),
    ])

arr =np.array([ 4,  1, 14,  8,  0,  6,  7,  7, 18,  3,  
                               3, 14,  9, 18,  7, 11,  3,  9,  7, 11,
                               6, 11,  5, 10,  7,  6, 13, 15,  3, 15,  
                               0, 11,  1, 10, 12, 14, 16,  9, 11,  5, 
                               5, 19,  8,  8, 15, 13, 14, 17, 18, 10, 
                               16, 4, 17,  4,  2,  0, 17,  4, 18, 17, 
                               10, 3,  2, 12, 12, 16, 12,  1,  9, 19,  
                               2, 10,  0,  1, 16, 12,  9, 13, 15, 13, 
                              16, 19,  2,  4,  6, 19,  5,  5,  8, 19, 
                              18,  1,  2, 15,  6,  0, 17,  8, 14, 13])


new_arr = np.array( [
    0, # Aquatic mammals -> Group 1: Aquatic Life
    0, # Fish -> Group 1: Aquatic Life
    3, # Flowers -> Group 4: Outdoor and Indoor Nature
    7, # Food containers -> Group 8: Containers and Contents
    4, # Fruit and vegetables -> Group 5: People and Their Food
    5, # Household electrical devices -> Group 6: Household and Communication
    5, # Household furniture -> Group 6: Household and Communication
    2, # Insects -> Group 3: Small Animals and Insects
    1, # Large carnivores -> Group 2: Domestic and Wild Animals
    9, # Large man-made outdoor things -> Group 10: Outdoor Structures and Scenes
    9, # Large natural outdoor scenes -> Group 10: Outdoor Structures and Scenes
    1, # Large omnivores and herbivores -> Group 2: Domestic and Wild Animals
    8, # Medium-sized mammals -> Group 9: Wild and Unusual Creatures
    8, # Non-insect invertebrates -> (not specifically grouped, as it was missed in the assignment, consider adjusting groups if needed)
    4, # People -> Group 5: People and Their Food
    8, # Reptiles -> Group 9: Wild and Unusual Creatures
    2, # Small mammals -> Group 3: Small Animals and Insects
    3, # Trees -> Group 4: Outdoor and Indoor Nature
    6, # Vehicles 1 -> Group 7: Transportation Types
    6  # Vehicles 2 -> Group 7: Transportation Types
]
)
old_lbls = list(range(100))
old2new = {i:new_arr[arr[i]] for i in range(len(arr))}
new_lbls = list(np.unique(new_arr))
count = len(new_lbls)


arr =[ 4,  1, 14,  8,  0,  6,  7,  7, 18,  3,  
                               3, 14,  9, 18,  7, 11,  3,  9,  7, 11,
                               6, 11,  5, 10,  7,  6, 13, 15,  3, 15,  
                               0, 11,  1, 10, 12, 14, 16,  9, 11,  5, 
                               5, 19,  8,  8, 15, 13, 14, 17, 18, 10, 
                               16, 4, 17,  4,  2,  0, 17,  4, 18, 17, 
                               10, 3,  2, 12, 12, 16, 12,  1,  9, 19,  
                               2, 10,  0,  1, 16, 12,  9, 13, 15, 13, 
                              16, 19,  2,  4,  6, 19,  5,  5,  8, 19, 
                              18,  1,  2, 15,  6,  0, 17,  8, 14, 13]

classes_2_keep = [18,19,3, 5, 6]

labels_2_keep = [idx for idx, el in enumerate(arr) if el in classes_2_keep] 

transform = torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(32, scale=(0.08, 1)),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ColorJitter(0.4, 0.4, 0.4, 0.4),
        torchvision.transforms.RandomGrayscale(p=0.2),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.44087801806139126, 0.42790631331699347, 0.3867879370752931),
            (0.26826768628079806, 0.2610450402318512, 0.26866836876860795),
        ),
    ])
"""

old_lbls = list(range(100))
labels_2_keep = list(range(25))
# labels_2_keep = [0,1,2,3]

old2new = {}
count = 0
for old_lbl in old_lbls:
    if old_lbl in labels_2_keep: 
        old2new[old_lbl] = count
        count += 1

for old_lbl in old_lbls:
    if old_lbl not in labels_2_keep: 
        old2new[old_lbl] = count

new_lbls = list(range(count+1))


def get_data_loader(opt):
    transform = torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(32, scale=(0.08, 1)),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ColorJitter(0.4, 0.4, 0.4, 0.4),
        torchvision.transforms.RandomGrayscale(p=0.2),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.44087801806139126, 0.42790631331699347, 0.3867879370752931),
            (0.26826768628079806, 0.2610450402318512, 0.26866836876860795),
        ),
    ])
    dataset = TwoAugUnsupervisedDatasetLbl(
        torchvision.datasets.CIFAR100(opt.data_folder, 'train', download=True), 
        transform=transform, 
        lblmap=old2new )
    
    return torch.utils.data.DataLoader(dataset, batch_size=opt.batch_size, num_workers=opt.num_workers,
                                       shuffle=True, pin_memory=True)

In [13]:
encoder = SmallAlexNet(feat_dim=opt.feat_dim, cifar=True).to(opt.gpus[0])


In [53]:
print(f'Optimize: {opt.align_w:g} * loss_align(alpha={opt.align_alpha:g}) + {opt.unif_w:g} * loss_uniform(t={opt.unif_t:g})')

torch.cuda.set_device(opt.gpus[0])
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

encoder = SmallAlexNet(feat_dim=opt.feat_dim, cifar=True).to(opt.gpus[0])

optim = torch.optim.Adam(encoder.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optim, gamma=opt.lr_decay_rate,
                                                 milestones=opt.lr_decay_epochs)

loader = get_data_loader(opt)
align_meter = AverageMeter('align_loss')
unif_meter = AverageMeter('uniform_loss')
loss_meter = AverageMeter('total_loss')
it_time_meter = AverageMeter('iter_time')

for epoch in range(opt.epochs):
    align_meter.reset()
    unif_meter.reset()
    loss_meter.reset()
    it_time_meter.reset()
    t0 = time.time()
    for ii, (im_x, im_y, lbl) in enumerate(loader):
        optim.zero_grad()
        x, y = encoder(torch.cat([im_x.to(opt.gpus[0]), im_y.to(opt.gpus[0])])).chunk(2)
        
        align_loss_val = align_loss(x, y, alpha=opt.align_alpha)
        # group according to new_lbls

        z = torch.cat( [x, y])
        lbl_z = torch.cat([lbl, lbl])
        unif_losses = torch.cat([uniform_loss_prelog(z[lbl_z==new_lbl]) for new_lbl in new_lbls])
        unif_loss_val = torch.log( torch.mean(unif_losses) )
        
        loss = align_loss_val * opt.align_w + unif_loss_val * opt.unif_w
        align_meter.update(align_loss_val, x.shape[0])
        unif_meter.update(unif_loss_val)
        loss_meter.update(loss, x.shape[0])
        loss.backward()
        optim.step()
        it_time_meter.update(time.time() - t0)
        if ii % opt.log_interval == 0:
            print(f"Epoch {epoch}/{opt.epochs}\tIt {ii}/{len(loader)}\t" +
                  f"{align_meter}\t{unif_meter}\t{loss_meter}\t{it_time_meter}")
        t0 = time.time()
    scheduler.step()

ckpt_file = os.path.join(opt.save_folder, 'encoder.pth')
torch.save(encoder.state_dict(), ckpt_file)
print(f'Saved to {ckpt_file}')

Optimize: 1 * loss_align(alpha=2) + 1 * loss_uniform(t=2)
Files already downloaded and verified
Epoch 0/200	It 0/66	align_loss 1.224504 (1.224504)	uniform_loss -2.548979 (-2.548979)	total_loss -1.324475 (-1.324475)	iter_time 3.744576 (3.744576)
Epoch 0/200	It 40/66	align_loss 0.538808 (0.331966)	uniform_loss -1.128203 (-0.701110)	total_loss -0.589394 (-0.369144)	iter_time 0.153217 (0.239226)
Epoch 1/200	It 0/66	align_loss 0.699515 (0.699515)	uniform_loss -1.626389 (-1.626389)	total_loss -0.926874 (-0.926874)	iter_time 4.069695 (4.069695)
Epoch 1/200	It 40/66	align_loss 0.870620 (0.784414)	uniform_loss -2.231381 (-1.948607)	total_loss -1.360761 (-1.164193)	iter_time 0.153626 (0.247472)
Epoch 2/200	It 0/66	align_loss 0.927554 (0.927554)	uniform_loss -2.364950 (-2.364950)	total_loss -1.437396 (-1.437396)	iter_time 4.500770 (4.500770)
Epoch 2/200	It 40/66	align_loss 0.950227 (0.877513)	uniform_loss -2.494799 (-2.420540)	total_loss -1.544572 (-1.543027)	iter_time 0.154047 (0.257995)
Epoch 3

Supervised Training

In [18]:
def get_data_loader_sup(opt):
    transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(70),
        torchvision.transforms.CenterCrop(32),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.44087801806139126, 0.42790631331699347, 0.3867879370752931),
            (0.26826768628079806, 0.2610450402318512, 0.26866836876860795),
        ),
    ])
    dataset = TwoAugUnsupervisedDatasetLbl(
        torchvision.datasets.CIFAR100(opt.data_folder, 'train', download=True), 
        transform=transform, 
        lblmap=old2new )
    
    return torch.utils.data.DataLoader(dataset, batch_size=opt.batch_size, num_workers=opt.num_workers,
                                       shuffle=True, pin_memory=True)

In [54]:
import itertools

In [55]:
encoder = SmallAlexNet(feat_dim=opt.feat_dim, cifar=True).to(opt.gpus[0])

n_classes = 100 if True else 10
classifier = torch.nn.Linear(opt.feat_dim, n_classes).to(opt.gpus[0])
optim = torch.optim.Adam( itertools.chain(classifier.parameters(), encoder.parameters()), lr=1e-2 )

scheduler = torch.optim.lr_scheduler.MultiStepLR(optim, gamma=opt.lr_decay_rate, milestones=opt.lr_decay_epochs)

loader = get_data_loader_sup(opt)

loss_meter = AverageMeter('total_loss')
it_time_meter = AverageMeter('iter_time')
for epoch in range(opt.epochs):
    loss_meter.reset()
    it_time_meter.reset()
    t0 = time.time()
    for ii, (im_x, im_y, lbl) in enumerate(loader):
        optim.zero_grad()
    
        x = encoder(im_x.to(opt.gpus[0]))
        # group according to new_lbls
        loss = torch.nn.functional.cross_entropy( classifier(x), lbl.to(opt.gpus[0]))
        loss_meter.update(loss, x.shape[0])
        loss.backward()
        optim.step()
        it_time_meter.update(time.time() - t0)

        if ii % opt.log_interval == 0:
            print(f"Epoch {epoch}/{opt.epochs}\tIt {ii}/{len(loader)}\t" +
                  f"{loss_meter}\t{it_time_meter}")
        t0 = time.time()

    scheduler.step()
ckpt_file = os.path.join(opt.save_folder, 'encoder.pth')
torch.save(encoder.state_dict(), ckpt_file)
print(f'Saved to {ckpt_file}')

Files already downloaded and verified
Epoch 0/200	It 0/66	total_loss 4.676263 (4.676263)	iter_time 8.411062 (8.411062)
Epoch 0/200	It 40/66	total_loss 1.712299 (2.817281)	iter_time 0.054871 (0.258113)
Epoch 1/200	It 0/66	total_loss 1.369541 (1.369541)	iter_time 5.113275 (5.113275)
Epoch 1/200	It 40/66	total_loss 1.360537 (1.408408)	iter_time 0.056095 (0.186002)
Epoch 2/200	It 0/66	total_loss 1.414740 (1.414740)	iter_time 5.587875 (5.587875)
Epoch 2/200	It 40/66	total_loss 1.280729 (1.352436)	iter_time 0.055602 (0.189638)
Epoch 3/200	It 0/66	total_loss 1.320413 (1.320413)	iter_time 5.422179 (5.422179)
Epoch 3/200	It 40/66	total_loss 1.329554 (1.300714)	iter_time 0.057104 (0.186666)
Epoch 4/200	It 0/66	total_loss 1.272256 (1.272256)	iter_time 5.371913 (5.371913)
Epoch 4/200	It 40/66	total_loss 1.286243 (1.253529)	iter_time 0.055948 (0.184197)
Epoch 5/200	It 0/66	total_loss 1.231850 (1.231850)	iter_time 5.017498 (5.017498)
Epoch 5/200	It 40/66	total_loss 1.239444 (1.201574)	iter_time 0.05

In [71]:
"""
    Here we  do the linear evaluation, the old labels are provided to the linear objective as one hot
"""
import time
import argparse

import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F

from util import AverageMeter
from encoder import SmallAlexNet
MODEL_2_LOAD="./results/base_final_cifar100_100_resize_transform_align1alpha2_unif1t2_iter0/encoder.pth" # unsupervised
MODEL_2_LOAD="./results/cifar100_100_25_155_align1alpha2_unif1t2_iter0/encoder.pth" # cond unsupervised

MODEL_2_LOAD="./results/base_final_cifar100_200_resize_transform_align1alpha2_unif1t2_iter0/encoder.pth" # unsupervised "Best validation accuracy 0.9586"
MODEL_2_LOAD="./results/cifar100_200_25_155_align1alpha2_unif1t2_iter0/encoder.pth" # cond unsupervised Best validation accuracy 0.96844
MODEL_2_LOAD="./results/cifar100_series_supervised_0_200/encoder.pth" # supervised Best validation accuracy 0.59464
MODEL_2_LOAD="./results/cifar100_200_50labels_align1alpha2_unif1t2_iter0/encoder.pth"
MODEL_2_LOAD="../old_exp_1/results/base_final_cifar100_200_resize_transform_align1alpha2_unif1t2_iter0/encoder.pth"

MODEL_2_LOAD="./results/cifar100_200_coarse_align1alpha2_unif1t2_iter0/encoder.pth" # Best validation accuracy 0.98148
MODEL_2_LOAD="./results/cifar100_series_coarse_unsupervised_0_200/encoder.pth"
MODEL_2_LOAD="./results/cifar100_unsupervisedcond_32_0_200/encoder.pth"
MODEL_2_LOAD="./results/cifar100_series_unsupervisedcond_0_200/encoder.pth"
MODEL_2_LOAD="./results/cifar100_unsupervised_32_0_200/unsupervised_encoder_32.pth"
MODEL_2_LOAD="./results/cifar100_unsupervisedcond_smllr_32_0_200/encoder.pth"


MODEL_2_LOAD="./results/cifar100_100_newsupervised_sideinformation_align1alpha2_unif1t2_iter0/encoder.pth"
MODEL_2_LOAD="./results/cifar100_unsupervisedcond_newlbls_smllr_32_0_200/encoder.pth"
MODEL_2_LOAD="./results/cifar100_unsupervised_smllr_32_0_200/encoder"
MODEL_2_LOAD="./results/cifar100_supervised_newlbls_smllr_32_0_200/encoder.pth"

MODEL_2_LOAD="./results/cifar100_supervised_oldnewnewset_smllr_32_0_200/encoder.pth"
MODEL_2_LOAD="./results/cifar100_supervised_oldnewnewset_smllr_128_0_200/encoder.pth"
MODEL_2_LOAD="./results/cifar100_unsupervised_smllr_128_0_200/encoder"
MODEL_2_LOAD="./results/cifar100_unsupervised_smllr_128_0_200/encoder"
MODEL_2_LOAD="./results/cifar100_unsupervised_smllr_128_0_200/encoder"
MODEL_2_LOAD="./results/cifar100_175_unsupervised_smllr_128_0_200/encoder"
MODEL_2_LOAD="./results/cifar100_supervised_170__oldnewnewset_smllr_128_0_200/encoder.pth"
MODEL_2_LOAD="./results/cifar100_supervised_170_2_oldnewnewset_smllr_128_0_200/encoder.pth"
MODEL_2_LOAD="./results/cifar100_supervised_170_2_oldnewnewset_smllr_128_0_200/encoder.pth"
MODEL_2_LOAD="./results/cifar100_175_unsupervised_smllr_128_2_200/encoder"
MODEL_2_LOAD="./results/cifar100_175_unsupervised_smllr_128_4_200/encoder"
MODEL_2_LOAD="./results/cifar100_175_unsupervised_smllr_128_9_200/encoder"
USE_MOD_LBL = True
encoder= SmallAlexNet(feat_dim=128, cifar=True).to(opt.gpus[0])
encoder.load_state_dict(torch.load(MODEL_2_LOAD))


<All keys matched successfully>

In [72]:
def parse_option():
    parser = argparse.ArgumentParser('STL-10 Representation Learning with Alignment and Uniformity Losses')

    parser.add_argument('--encoder_checkpoint', type=str, help='Encoder checkpoint to evaluate', default=MODEL_2_LOAD)
    parser.add_argument('--feat_dim', type=int, default=128, help='Encoder feature dimensionality')
    parser.add_argument('--layer_index', type=int, default=-2, help='Evaluation layer')

    parser.add_argument('--batch_size', type=int, default=128, help='Batch size')
    parser.add_argument('--epochs', type=int, default=100, help='Number of training epochs')
    parser.add_argument('--lr', type=float, default=1e-3, help='Learning rate')
    parser.add_argument('--lr_decay_rate', type=float, default=0.2, help='Learning rate decay rate')
    parser.add_argument('--lr_decay_epochs', type=str, default='60,80', help='When to decay learning rate')

    parser.add_argument('--num_workers', type=int, default=6, help='Number of data loader workers to use')
    parser.add_argument('--log_interval', type=int, default=40, help='Number of iterations between logs')
    parser.add_argument('--gpus', default=[0], nargs='*', type=int,
                        help='List of GPU indices to use, e.g., --gpus 0 1 2 3')

    parser.add_argument('--data_folder', type=str, default='./data', help='Path to data')

    opt = parser.parse_args("")

    if opt.lr is None:
        opt.lr = 0.12 * (opt.batch_size / 256)

    opt.gpu = torch.device('cuda', opt.gpus[0])
    opt.lr_decay_epochs = list(map(int, opt.lr_decay_epochs.split(',')))

    return opt


class DatasetModifiedLbl(torch.utils.data.Dataset):
    r"""Returns two augmentation and no labels."""

    def __init__(self, dataset, lblmap=None):
        self.dataset = dataset
        self.lblmap = copy.deepcopy(lblmap)

    def __getitem__(self, index):
        image, lbl = self.dataset[index]
        lbl2return = lbl if self.lblmap is None else self.lblmap[lbl]
        return image, lbl2return

    def __len__(self):
        return len(self.dataset)

In [73]:
class DatasetModifiedLblandLbl(torch.utils.data.Dataset):
    r"""Returns two augmentation and no labels."""

    def __init__(self, dataset, lblmap):
        self.dataset = dataset
        self.lblmap = copy.deepcopy(lblmap)

    def __getitem__(self, index):
        image, lbl = self.dataset[index]
        return image, self.lblmap[lbl], lbl

    def __len__(self):
        return len(self.dataset)

In [74]:
class DatasetSubset(torch.utils.data.Dataset):
    def __init__(self, dataset, lbl2keep):
        self.lbl_map = {lbl:lbl_idx for lbl_idx, lbl in enumerate(lbl2keep)}
        self.dataset = [(datum,self.lbl_map[lbl]) for datum,lbl in dataset if lbl in lbl2keep]
    def __getitem__(self, index):
        return self.dataset[index]
    def __len__(self):
        return len(self.dataset)

In [75]:
def get_data_loaders(opt):
    train_transform = torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(32, scale=(0.08, 1)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.44087801806139126, 0.42790631331699347, 0.3867879370752931),
            (0.26826768628079806, 0.2610450402318512, 0.26866836876860795),
        ),
        torchvision.transforms.RandomHorizontalFlip()
    ])
    val_transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(35),
        torchvision.transforms.CenterCrop(32),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.44087801806139126, 0.42790631331699347, 0.3867879370752931),
            (0.26826768628079806, 0.2610450402318512, 0.26866836876860795),
        ),
    ])
    
    train_dataset = DatasetModifiedLblandLbl( torchvision.datasets.CIFAR100(opt.data_folder, 'train', download=True, transform=train_transform), lblmap=old2new )
    val_dataset =  DatasetModifiedLblandLbl( torchvision.datasets.CIFAR100(opt.data_folder, 'test', transform=val_transform), lblmap=old2new)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=opt.batch_size,
                                               num_workers=opt.num_workers, shuffle=True, pin_memory=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=opt.batch_size,
                                             num_workers=opt.num_workers, pin_memory=True)
    return train_loader, val_loader


def validate_comb(opt, encoder, classifier, val_loader):
    correct = 0
    with torch.no_grad():
        for images, labels_mod, labels_act in val_loader:
            pred = classifier(torch.cat( (encoder(images.to(opt.gpus[0]), layer_index=opt.layer_index), torch.nn.functional.one_hot(labels_mod.to(opt.gpus[0]), num_classes=len(new_lbls))), dim=1)).argmax(dim=1)
            correct += (pred.cpu() == labels_act).sum().item()
    return correct / len(val_loader.dataset)

def validate(opt, encoder, classifier, val_loader):
    correct = 0
    with torch.no_grad():
        for images, labels_act in val_loader:
            pred = classifier( encoder(images.to(opt.gpus[0]), layer_index=opt.layer_index).flatten(1) ).argmax(dim=1)
            correct += (pred.cpu() == labels_act).sum().item()
    return correct / len(val_loader.dataset)

In [76]:
opt=parse_option()
opt.gpu=opt.gpus[0]
torch.cuda.set_device(opt.gpus[0])
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True


encoder.eval()
train_loader, val_loader = get_data_loaders(opt)

with torch.no_grad():
    sample, _ = train_loader.dataset.dataset[0]
    eval_numel = encoder(sample.unsqueeze(0).to(opt.gpus[0]), layer_index=opt.layer_index).numel()
print(f'Feature dimension: {eval_numel}')

Files already downloaded and verified
Feature dimension: 4096


In [84]:
import glob

In [85]:
accs_unsupervised = []

In [86]:
for i in range(10):
    MODEL_2_LOAD = os.path.join("./","results", f"cifar100_175_unsupervised_smllr_128_{i}_200", "encoder")
    print(f"loading {MODEL_2_LOAD}")
    encoder.load_state_dict(torch.load(MODEL_2_LOAD))
    encoder = encoder.to(opt.gpus[0])

    encoder.load_state_dict(torch.load(MODEL_2_LOAD))

    encoder = encoder.to(opt.gpus[0])


    classifier =  nn.Linear( eval_numel + len(new_lbls), len(old_lbls)).to(opt.gpus[0])

    optim = torch.optim.Adam(classifier.parameters(), lr=opt.lr, betas=(0.5, 0.999))
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optim, gamma=opt.lr_decay_rate,
                                                     milestones=opt.lr_decay_epochs)

    val_accs = []
    loss_meter = AverageMeter('loss')
    it_time_meter = AverageMeter('iter_time')
    for epoch in tqdm(range(opt.epochs)):
        loss_meter.reset()
        it_time_meter.reset()
        t0 = time.time()
        for ii, (images, labels_mod, labels) in enumerate(train_loader):
            optim.zero_grad()
            labels_mod_onehot = torch.nn.functional.one_hot(labels_mod, num_classes=len(new_lbls)).to(opt.gpus[0])
            with torch.no_grad():
                feats = encoder(images.to(opt.gpus[0]), layer_index=opt.layer_index).flatten(1)


            logits = classifier(torch.cat((feats, labels_mod_onehot), dim=1))

            loss = F.cross_entropy(logits, labels.to(opt.gpus[0]))
            loss_meter.update(loss, images.shape[0])
            loss.backward()
            optim.step()
            it_time_meter.update(time.time() - t0)
            if ii % opt.log_interval == 0:
                print(f"Epoch {epoch}/{opt.epochs}\tIt {ii}/{len(train_loader)}\t{loss_meter}\t{it_time_meter}")
            t0 = time.time()
        scheduler.step()
        val_acc = validate_comb(opt,encoder,classifier,val_loader)
        val_accs.append(val_acc)
        print(f"Epoch {epoch}/{opt.epochs}\tval_acc {val_acc*100:.4g}%")
    print(f"Best validation accuracy {max(val_accs)}")
    accs_unsupervised.append(val_accs)

loading ./results/cifar100_175_unsupervised_smllr_128_0_200/encoder


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 0/100	It 0/391	loss 4.623976 (4.623976)	iter_time 0.286441 (0.286441)
Epoch 0/100	It 40/391	loss 3.008940 (3.551139)	iter_time 0.004481 (0.014086)
Epoch 0/100	It 80/391	loss 2.779517 (3.149994)	iter_time 0.004417 (0.010838)
Epoch 0/100	It 120/391	loss 2.260803 (2.938166)	iter_time 0.017423 (0.009704)
Epoch 0/100	It 160/391	loss 2.473135 (2.800270)	iter_time 0.009140 (0.008932)
Epoch 0/100	It 200/391	loss 2.363260 (2.708774)	iter_time 0.004392 (0.008660)
Epoch 0/100	It 240/391	loss 2.046580 (2.634561)	iter_time 0.004361 (0.008475)
Epoch 0/100	It 280/391	loss 2.358630 (2.580361)	iter_time 0.004439 (0.008252)
Epoch 0/100	It 320/391	loss 1.999617 (2.529749)	iter_time 0.004878 (0.008220)
Epoch 0/100	It 360/391	loss 2.269775 (2.487554)	iter_time 0.015575 (0.008058)


  1%|          | 1/100 [00:05<09:04,  5.50s/it]

Epoch 0/100	val_acc 57.28%
Epoch 1/100	It 0/391	loss 2.204614 (2.204614)	iter_time 0.269709 (0.269709)
Epoch 1/100	It 40/391	loss 1.793815 (1.988241)	iter_time 0.004515 (0.012626)
Epoch 1/100	It 80/391	loss 2.243593 (1.984231)	iter_time 0.021991 (0.010156)
Epoch 1/100	It 120/391	loss 1.842229 (1.971202)	iter_time 0.004367 (0.009187)
Epoch 1/100	It 160/391	loss 1.843166 (1.965823)	iter_time 0.004421 (0.008770)
Epoch 1/100	It 200/391	loss 1.674485 (1.960306)	iter_time 0.021328 (0.008532)
Epoch 1/100	It 240/391	loss 2.031726 (1.950641)	iter_time 0.004386 (0.008284)
Epoch 1/100	It 280/391	loss 1.841505 (1.940678)	iter_time 0.004588 (0.008196)
Epoch 1/100	It 320/391	loss 1.807971 (1.929035)	iter_time 0.015148 (0.008112)
Epoch 1/100	It 360/391	loss 1.752415 (1.925027)	iter_time 0.004638 (0.007950)


...

Epoch 97/100	val_acc 92.58%
Epoch 98/100	It 0/391	loss 0.815572 (0.815572)	iter_time 0.279612 (0.279612)
Epoch 98/100	It 40/391	loss 0.707330 (0.771110)	iter_time 0.004368 (0.013922)
Epoch 98/100	It 80/391	loss 0.692420 (0.805120)	iter_time 0.004323 (0.010717)
Epoch 98/100	It 120/391	loss 0.927362 (0.811592)	iter_time 0.021624 (0.009622)
Epoch 98/100	It 160/391	loss 1.021680 (0.810123)	iter_time 0.004431 (0.008883)
Epoch 98/100	It 200/391	loss 0.743290 (0.808876)	iter_time 0.004274 (0.008395)
Epoch 98/100	It 240/391	loss 0.915255 (0.808810)	iter_time 0.014564 (0.008057)
Epoch 98/100	It 280/391	loss 1.013645 (0.810172)	iter_time 0.004547 (0.007836)
Epoch 98/100	It 320/391	loss 0.942656 (0.810448)	iter_time 0.021651 (0.007795)
Epoch 98/100	It 360/391	loss 0.735205 (0.809346)	iter_time 0.004378 (0.007658)


 99%|█████████▉| 99/100 [09:01<00:05,  5.44s/it]

Epoch 98/100	val_acc 92.59%
Epoch 99/100	It 0/391	loss 0.690396 (0.690396)	iter_time 0.274461 (0.274461)
Epoch 99/100	It 40/391	loss 0.530566 (0.794337)	iter_time 0.004395 (0.013767)
Epoch 99/100	It 80/391	loss 0.889231 (0.795439)	iter_time 0.004413 (0.010644)
Epoch 99/100	It 120/391	loss 0.902603 (0.816163)	iter_time 0.020884 (0.009565)
Epoch 99/100	It 160/391	loss 0.863634 (0.821725)	iter_time 0.004432 (0.008912)
Epoch 99/100	It 200/391	loss 0.752901 (0.822438)	iter_time 0.004401 (0.008631)
Epoch 99/100	It 240/391	loss 0.742847 (0.819684)	iter_time 0.022236 (0.008425)
Epoch 99/100	It 280/391	loss 0.524829 (0.816540)	iter_time 0.004607 (0.008215)
Epoch 99/100	It 320/391	loss 0.638690 (0.815058)	iter_time 0.004681 (0.008095)
Epoch 99/100	It 360/391	loss 0.774403 (0.814078)	iter_time 0.004413 (0.007943)


100%|██████████| 100/100 [09:06<00:00,  5.47s/it]

Epoch 99/100	val_acc 92.6%
Best validation accuracy 0.92606


In [ ]:
Epoch 99/100	val_acc 92.17%
Best validation accuracy 0.92168
Epoch 99/100	val_acc 92.62%
Best validation accuracy 0.92642
Epoch 99/100	val_acc 94.19%
Best validation accuracy 0.94192
Epoch 99/100	val_acc 94.43%
Best validation accuracy 0.94444
Epoch 99/100	val_acc 92.71%
Best validation accuracy 0.92728


Epoch 99/100	val_acc 95.03%
Best validation accuracy 0.95042
Epoch 99/100	val_acc 93.07%
Best validation accuracy 0.93102
Epoch 99/100	val_acc 94.25%
Best validation accuracy 0.94264
Epoch 90/100	val_acc 94.47%


In [56]:
"""
    Here we  do the linear evaluation, the old labels are provided to the linear objective as one hot
"""
import time
import argparse

import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F

from util import AverageMeter
from encoder import SmallerAlexNet
MODEL_2_LOAD="./results/base_final_cifar100_100_resize_transform_align1alpha2_unif1t2_iter0/encoder.pth" # unsupervised
MODEL_2_LOAD="./results/cifar100_100_25_155_align1alpha2_unif1t2_iter0/encoder.pth" # cond unsupervised

MODEL_2_LOAD="./results/base_final_cifar100_200_resize_transform_align1alpha2_unif1t2_iter0/encoder.pth" # unsupervised "Best validation accuracy 0.9586"
MODEL_2_LOAD="./results/cifar100_200_25_155_align1alpha2_unif1t2_iter0/encoder.pth" # cond unsupervised Best validation accuracy 0.96844
MODEL_2_LOAD="./results/cifar100_series_supervised_0_200/encoder.pth" # supervised Best validation accuracy 0.59464
MODEL_2_LOAD="./results/cifar100_200_50labels_align1alpha2_unif1t2_iter0/encoder.pth"
MODEL_2_LOAD="../old_exp_1/results/base_final_cifar100_200_resize_transform_align1alpha2_unif1t2_iter0/encoder.pth"

MODEL_2_LOAD="./results/cifar100_200_coarse_align1alpha2_unif1t2_iter0/encoder.pth" # Best validation accuracy 0.98148
MODEL_2_LOAD="./results/cifar100_series_coarse_unsupervised_0_200/encoder.pth"
MODEL_2_LOAD="./results/cifar100_unsupervisedcond_32_0_200/encoder.pth"
MODEL_2_LOAD="./results/cifar100_series_unsupervisedcond_0_200/encoder.pth"
MODEL_2_LOAD="./results/cifar100_unsupervised_32_0_200/unsupervised_encoder_32.pth"
MODEL_2_LOAD="./results/cifar100_unsupervisedcond_smllr_32_0_200/encoder.pth"
MODEL_2_LOAD="./results/cifar100_unsupervised_smllr_32_0_200/encoder"

MODEL_2_LOAD="./results/cifar100_unsupervisedcond_yin_newlbls_smllr_32_0_200/encoder.pth"
MODEL_2_LOAD="./results/cifar100_unsupervisedcond_yin_newlbls_smllr_32_0_200/encoder.pth"

MODEL_2_LOAD="./results/cifar100_supervised_oldnewset_smllr_32_0_200/encoder.pth"
MODEL_2_LOAD="./results/cifar100_unsupervisedcond_yin_newlbls_smllr_32_0_200/encoder.pth"
MODEL_2_LOAD="./results/cifar100_supervised_oldnewnewset_smllr_32_0_200/encoder.pth"
MODEL_2_LOAD="./results/cifar100_unsupervised_smllr_32_0_200/encoder"


USE_MOD_LBL = True
encoder= SmallerAlexNet(feat_dim=32, cifar=True)
encoder.load_state_dict(torch.load(MODEL_2_LOAD))


FileNotFoundError: [Errno 2] No such file or directory: './results/cifar100_unsupervised_smllr_32_0_200/encoder'

In [57]:
encoder = encoder.to(opt.gpus[0])

In [58]:
def parse_option():
    parser = argparse.ArgumentParser('STL-10 Representation Learning with Alignment and Uniformity Losses')

    parser.add_argument('--encoder_checkpoint', type=str, help='Encoder checkpoint to evaluate', default=MODEL_2_LOAD)
    parser.add_argument('--feat_dim', type=int, default=128, help='Encoder feature dimensionality')
    parser.add_argument('--layer_index', type=int, default=-2, help='Evaluation layer')

    parser.add_argument('--batch_size', type=int, default=128, help='Batch size')
    parser.add_argument('--epochs', type=int, default=100, help='Number of training epochs')
    parser.add_argument('--lr', type=float, default=1e-3, help='Learning rate')
    parser.add_argument('--lr_decay_rate', type=float, default=0.2, help='Learning rate decay rate')
    parser.add_argument('--lr_decay_epochs', type=str, default='60,80', help='When to decay learning rate')

    parser.add_argument('--num_workers', type=int, default=6, help='Number of data loader workers to use')
    parser.add_argument('--log_interval', type=int, default=40, help='Number of iterations between logs')
    parser.add_argument('--gpus', default=[0], nargs='*', type=int,
                        help='List of GPU indices to use, e.g., --gpus 0 1 2 3')

    parser.add_argument('--data_folder', type=str, default='./data', help='Path to data')

    opt = parser.parse_args("")

    if opt.lr is None:
        opt.lr = 0.12 * (opt.batch_size / 256)

    opt.gpu = torch.device('cuda', opt.gpus[0])
    opt.lr_decay_epochs = list(map(int, opt.lr_decay_epochs.split(',')))

    return opt


class DatasetModifiedLbl(torch.utils.data.Dataset):
    r"""Returns two augmentation and no labels."""

    def __init__(self, dataset, lblmap=None):
        self.dataset = dataset
        self.lblmap = copy.deepcopy(lblmap)

    def __getitem__(self, index):
        image, lbl = self.dataset[index]
        lbl2return = lbl if self.lblmap is None else self.lblmap[lbl]
        return image, lbl2return

    def __len__(self):
        return len(self.dataset)

In [59]:
class DatasetModifiedLblandLbl(torch.utils.data.Dataset):
    r"""Returns two augmentation and no labels."""

    def __init__(self, dataset, lblmap):
        self.dataset = dataset
        self.lblmap = copy.deepcopy(lblmap)

    def __getitem__(self, index):
        image, lbl = self.dataset[index]
        return image, self.lblmap[lbl], lbl

    def __len__(self):
        return len(self.dataset)
    
class DatasetSubset(torch.utils.data.Dataset):
    def __init__(self, dataset, lbl2keep):
        self.lbl_map = {lbl:lbl_idx for lbl_idx, lbl in enumerate(lbl2keep)}
        self.dataset = [(datum,self.lbl_map[lbl]) for datum,lbl in dataset if lbl in lbl2keep]
    def __getitem__(self, index):
        return self.dataset[index]
    def __len__(self):
        return len(self.dataset)

In [60]:
def get_data_loaders(opt):
    train_transform = torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(32, scale=(0.08, 1)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.44087801806139126, 0.42790631331699347, 0.3867879370752931),
            (0.26826768628079806, 0.2610450402318512, 0.26866836876860795),
        ),
        torchvision.transforms.RandomHorizontalFlip()
    ])
    val_transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(35),
        torchvision.transforms.CenterCrop(32),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.44087801806139126, 0.42790631331699347, 0.3867879370752931),
            (0.26826768628079806, 0.2610450402318512, 0.26866836876860795),
        ),
    ])
    train_dataset = DatasetModifiedLblandLbl( torchvision.datasets.CIFAR100(opt.data_folder, 'train', download=True, transform=train_transform), lblmap=old2new)
    val_dataset =  DatasetModifiedLblandLbl( torchvision.datasets.CIFAR100(opt.data_folder, 'test', transform=val_transform), lblmap=old2new)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=opt.batch_size,
                                               num_workers=opt.num_workers, shuffle=True, pin_memory=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=opt.batch_size,
                                             num_workers=opt.num_workers, pin_memory=True)
    return train_loader, val_loader


def validate_comb(opt, encoder, classifier, val_loader):
    correct = 0
    with torch.no_grad():
        for images, labels_mod, labels_act in val_loader:
            labels_mod_onehot =torch.nn.functional.one_hot(labels_mod, num_classes=len(new_lbls)).type(torch.FloatTensor).to(opt.gpus[0])
            pred = classifier(torch.cat( (encoder(images.to(opt.gpus[0]),layer_index=opt.layer_index), labels_mod_onehot.to(opt.gpus[0])), dim=1)).argmax(dim=1)
            correct += (pred.cpu() == labels_act).sum().item()
    return correct / len(val_loader.dataset)

def validate(opt, encoder, classifier, val_loader):
    correct = 0
    with torch.no_grad():
        for images, labels_act in val_loader:
            labels_mod_onehot =  (len(new_lbls)-1) * torch.ones(images.shape[0]).type(torch.LongTensor).to(opt.gpus[0])
            pred = classifier( encoder(images.to(opt.gpus[0]), layer_index=opt.layer_index).flatten(1) ).argmax(dim=1)
            correct += (pred.cpu() == labels_act).sum().item()
    return correct / len(val_loader.dataset)

In [61]:
opt = parse_option()

In [62]:
opt.gpu=opt.gpus[0]
torch.cuda.set_device(opt.gpus[0])
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

# encoder.eval()
# valset_lbl2keep =[i for i in range(100) if i not in labels_2_keep] 

train_loader, val_loader = get_data_loaders(opt)

Files already downloaded and verified


In [63]:
lbl_mod

NameError: name 'lbl_mod' is not defined

In [ ]:
with torch.no_grad():
    sample, lbl_mod, lbl = next(iter(train_loader))
    # labels_mod_onehot = ( (len(new_lbls) -1 ) * torch.ones(sample.shape[0]) ).type(torch.LongTensor).to(opt.gpus[0])
    eval_numel = int(encoder(sample.to(opt.gpus[0]), layer_index=-2).numel() / sample.shape[0])
print(f'Feature dimension: {eval_numel}')

In [ ]:
opt.lr

In [64]:
classifier = nn.Linear(eval_numel + len(new_lbls), 100).to(opt.gpus[0])


val_accs = []
loss_meter = AverageMeter('loss')
it_time_meter = AverageMeter('iter_time')

epoch = 0

In [65]:

optim = torch.optim.Adam(classifier.parameters(), lr=opt.lr, betas=(0.5, 0.999))
scheduler = torch.optim.lr_scheduler.MultiStepLR(optim, gamma=opt.lr_decay_rate,
                                                 milestones=opt.lr_decay_epochs)

while epoch < opt.epochs:
    loss_meter.reset()
    it_time_meter.reset()
    t0 = time.time()
    for ii, (images, labels_mod, labels) in enumerate(train_loader):
        optim.zero_grad()
        with torch.no_grad():
            feats = encoder(images.to(opt.gpus[0]),layer_index=opt.layer_index).flatten(1)

       
        logits = classifier( torch.cat( (feats, torch.nn.functional.one_hot(labels_mod, num_classes=len(new_lbls)).type(torch.FloatTensor).to(opt.gpus[0])  ), dim=1 ))
        
        loss = F.cross_entropy(logits, labels.to(opt.gpus[0]))
        loss_meter.update(loss, images.shape[0])
        loss.backward()
        optim.step()
        it_time_meter.update(time.time() - t0)
        if ii % opt.log_interval == 0:
            print(f"Epoch {epoch}/{opt.epochs}\tIt {ii}/{len(train_loader)}\t{loss_meter}\t{it_time_meter}")
        t0 = time.time()
    scheduler.step()
    val_acc = validate_comb(opt,encoder,classifier,val_loader) if USE_MOD_LBL else validate(opt, encoder, classifier, val_loader) 
    val_accs.append(val_acc)
    
    epoch += 1
    print(f"Epoch {epoch}/{opt.epochs}\tval_acc {val_acc*100:.4g}%")
    
print(f"Best validation accuracy {max(val_accs)}")

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [55]:
os.path.dirname(opt.encoder_checkpoint)

'./results/cifar100_175_unsupervised_smllr_128_9_200'

In [53]:
import os

In [ ]:
os.path.ba